# DATA FOUNDATION

In [ ]:
import pandas as pd
import numpy as np 
import os
import gc # Để giải phóng RAM

BASE_DIR = "/MALLORN-Astronomical-Classification-Challenge/data/raw"

R_COEFFS = {'u': 4.239, 'g': 3.303, 'r': 2.285, 'i': 1.698, 'z': 1.263, 'y': 1.086}

In [ ]:
def load_log(base_dir, mode='train'):
    log_path = os.path.join(base_dir, f"{mode}_log.csv")
    if not os.path.exists(log_path):
        raise FileNotFoundError(f"Không tìm thấy file {mode}_log.csv")
    
    df_log=pd.read_csv(log_path)
    if mode == 'test':
        if 'target' not in df_log.columns:
            df_log['target'] = np.nan
        if 'Z_err' in df_log.columns:
            df_log['Z_err'] = 0
    return df_log

In [ ]:
def clean_data(df):
    n_original = len(df)

    # 1. Xứ lý NaN
    # Nếu Flux hoặc Flux_err bị NaN -> Xóa dòng vì dữ liệu vô nghĩa
    df = df.dropna(subset=['Flux', 'Flux_err', 'Flux_corrected', 'Flux_err_corrected'])

    # Nếu EBV bị NaN, điền bằng 0 (coi như không có bụi) để tránh lỗi tính toán
    if 'EBV' in df.columns:
        df['EBV'] = df['EBV'].fillna(0)

    # 2. Xử lý Flux âm
    # Không xóa FLux âm nhưng kiểm kê nó
    n_negative = (df['Flux_corrected'] < 0).sum()

    # 3. Sắp xếp thời gian
    df = df.sort_values(by=['object_id', 'Time (MJD)'], ascending=[True, True])
    df = df.reset_index(drop=True)

    n_dropped = n_original - len(df)
    if n_dropped > 0:
        print(f"   Đã loại bỏ {n_dropped} dòng chứa NaN.")
    if n_negative > 0:
        print(f"   Lưu ý: Có {n_negative} điểm đo có Flux Âm (vẫn giữ lại).")
    
    return df

In [ ]:
def process_one_split(split_name, df_log, base_dir, mode='train'):
    print(f"\n Đang xử lý: {split_name}...")
    
    lc_path = os.path.join(base_dir, split_name, f"{mode}_full_lightcurves.csv")
    
    df_meta_split = df_log[df_log['split'] == split_name].copy()
    
    df_lc = pd.read_csv(lc_path)

    df_lc_merged = df_lc.merge(
        df_meta_split[['object_id', 'EBV', 'target']],
        on='object_id',
        how='inner'
    )

    df_lc_merged['R_factor'] = df_lc_merged['Filter'].map(R_COEFFS)
    correction = 10 ** (0.4 * df_lc_merged['R_factor'] * df_lc_merged['EBV'])
    df_lc_merged['Flux_corrected'] = df_lc_merged['Flux'] * correction
    df_lc_merged['Flux_err_corrected'] = df_lc_merged['Flux_err'] * correction

    df_lc_clean = clean_data(df_lc_merged)
    del df_lc, df_lc_merged
    gc.collect()

    print(f"   -> Hoàn tất {split_name}")
    return df_lc_clean

# FEATURE ENGINEERING

STATISTICAL FEATURES

In [ ]:
def generate_statistical_features(df):
    print("Đang tạo: Statistical & Percentiles Features....")

    # 1. Định nghĩa các hàm thống kê
    aggregations = {
        'Flux_corrected': [
            'mean', 'std', 'max', 'min',
            ('q05', lambda x: x.quantile(0.05)), # Phân vị 5% (thay cho Min để tránh nhiễu)
            ('q25', lambda x: x.quantile(0.25)),
            ('q75', lambda x: x.quantile(0.75)),
            ('q95', lambda x: x.quantile(0.95)), # Phân vị 95% (thay cho Max để tránh nhiễu)
            'skew', # Độ lệch phân phối (quan trọng cho TDE)
            'count' # Số lượng điểm đo
        ],
        'Flux': ['max'], # Giữ lại max flux gốc để tham chiếu
        'Flux_err': ['mean'] # Sai số trung bình (đánh giá chất lượng đo)
    }

    # 2. Groupby và Aggregation (Thực hiện song song cho tất cả object)
    # Biến đổi bảng Long-fomr thành bảng thống kê sơ bộ
    df_agg = df.groupby(['object_id', 'Filter']).agg(aggregations)

    # 3. Làm phảng MultiIndex Columns
    df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]

    # 4. Unstack (Xoay trục Filter thành cột)
    df_features = df_agg.unstack('Filter')
    df_features.columns = [f"{filter_name}_{feature}" for feature, filter_name in df_features.columns]

    # 5. Tính các đặc trưng "Global" (Tổng hợp trên mọi band)
    global_agg = df.groupby('object_id')['Flux_corrected'].agg([
        ('global_max', 'max'),
        ('global_mean', 'mean'),
        ('global_std', 'std')
    ])

    # Ghép global vào bảng features
    df_features = df_features.join(global_agg)

    # 6. Xử lý NaN sinh ra do Unstack
    # (Ví dụ: Object A không có dữ liệu filter 'u', sẽ sinh ra NaN tại các cột u_...)
    # Chiến thuật: 
    # - Với Mean, Max, Min, Quantile: Fill = 0 (coi như tối đen)
    # - Với Count: Fill = 0
    df_features = df_features.fillna(0)

    print(f"✅ Hoàn thành Nhóm 1! Kích thước: {df_features.shape}")
    return df_features

MORPHOLOGY FEATURES

In [ ]:
def generate_morphology_features(df_clean, df_features_g1):
    print("   Đang tạo: Morphology Features")

    # index của dòng có Flux lớn nhất cho mỗi object/filter
    idx_max = df_clean.groupby(['object_id', 'Filter'])['Flux_corrected'].idxmax()

    # index của dòng đầu tiên và dòng cuối cùng (theo thời gian)
    idx_first = df_clean.groupby(['object_id', 'Filter'])['Time (MJD)'].idxmin()
    idx_last = df_clean.groupby(['object_id', 'Filter'])['Time (MJD)'].idxmax()

     # Trích xuất dữ liệu tại các điểm mốc
    df_peaks = df_clean.loc[idx_max][['object_id', 'Filter', 'Time (MJD)', 'Flux_corrected']].set_index(['object_id', 'Filter'])
    df_starts = df_clean.loc[idx_first][['object_id', 'Filter', 'Time (MJD)', 'Flux_corrected']].set_index(['object_id', 'Filter'])
    df_ends = df_clean.loc[idx_last][['object_id', 'Filter', 'Time (MJD)', 'Flux_corrected']].set_index(['object_id', 'Filter'])

    df_peaks.columns = ['Time_peak', 'Flux_peak']
    df_starts.columns = ['Time_start', 'Flux_start']
    df_ends.columns = ['Time_end', 'Flux_end']

    # Ghép lại thành một bảng cho morphology
    df_morp = pd.concat([df_peaks, df_starts, df_ends], axis=1)

    # Tính Time, Slope
    df_morp['Rise_time'] = df_morp['Time_peak'] - df_morp['Time_start']
    df_morp['Fall_time'] = df_morp['Time_end'] - df_morp['Time_peak']
    df_morp['Rise_slope'] = (df_morp['Flux_peak'] - df_morp['Flux_start']) / (df_morp['Rise_time'] + 0.1)
    df_morp['Fall_slope'] = (df_morp['Flux_end'] - df_morp['Flux_peak']) / (df_morp['Fall_time'] + 0.1)
    
    cols_time_slope = ['Rise_time', 'Fall_time', 'Rise_slope', 'Fall_slope']
    df_morp_ts = df_morp[cols_time_slope].unstack('Filter')
    df_morp_ts.columns = [f"{filter_name}_{feat}" for feat, filter_name in df_morp_ts.columns]

    # Tính Percent Amplitude
    amp_features = []
    filters = ['u', 'g', 'r', 'i', 'z', 'y']
    
    for f in filters:
        col_max = f"{f}_Flux_corrected_max"
        col_min = f"{f}_Flux_corrected_min"
        col_mean = f"{f}_Flux_corrected_mean"

        amp_series = (df_features_g1[col_max] - df_features_g1[col_min]) / (df_features_g1[col_mean] + 1e-6)
        amp_df = pd.DataFrame({f"{f}_percent_amplitude": amp_series})
        amp_features.append(amp_df)
    df_morp_amp = pd.concat(amp_features, axis=1)

    # Tính Kurtosis
    df_morp_kurt = df_clean.groupby(['object_id', 'Filter'])['Flux_corrected'].apply(lambda x: x.kurt()).unstack('Filter')
    df_morp_kurt.columns = [f"{f}_kurtosis" for f in df_morp_kurt.columns]

    # Tính Stetson K
    mean_cols = [c for c in df_features_g1.columns if 'mean' in c and 'Flux_corrected' in c and 'global' not in c]
    df_means = df_features_g1[mean_cols].copy()
    df_means.columns = [c.split('_')[0] for c in df_means.columns]

    df_means_flat = df_means.stack().reset_index()
    df_means_flat.columns = ['object_id', 'Filter', 'mean_flux']
    df_stetson = pd.merge(df_clean, df_means_flat, on=['object_id', 'Filter'], how='left')

    df_stetson['delta'] = (df_stetson['Flux_corrected'] - df_stetson['mean_flux']) / (df_stetson['Flux_err_corrected'] + 1e-6)
    
    df_stetson['abs_delta'] = df_stetson['delta'].abs()
    df_stetson['delta_sq'] = df_stetson['delta'] ** 2

    g = df_stetson.groupby(['object_id', 'Filter'])
    mean_abs_delta = g['abs_delta'].mean()
    mean_delta_sq = g['delta_sq'].mean()
    stetson_k = mean_abs_delta / (np.sqrt(mean_delta_sq) + 1e-6)

    df_morp_stetson = stetson_k.unstack('Filter')
    df_morp_stetson.columns = [f"{f}_stetson_k" for f in df_morp_stetson.columns]


    # Tạo bảng morphology features
    df_morp_final = df_morp_ts.join([df_morp_amp, df_morp_kurt, df_morp_stetson])

    # Xử lý NaN nếu có
    df_morp_final = df_morp_final.fillna(0)

    print(f"Hoàn thành Nhóm 2! Kích thước: {df_morp_final.shape}")
    return df_morp_final

COLOR FEATURES

In [ ]:
def generate_color_features(df_features_g1):
    print("   Đang tạo: Color Features")

    filters = ['u', 'g', 'r', 'i', 'z', 'y']
    new_features = []

    for i in range(len(filters) - 1):
        f1 = filters[i]
        f2 = filters[i+1]

        col_f1_mean = f"{f1}_Flux_corrected_mean"
        col_f2_mean = f"{f2}_Flux_corrected_mean"

        ratio_mean = df_features_g1[col_f1_mean] / (df_features_g1[col_f2_mean] + 1e-6)
        diff_mean = df_features_g1[col_f1_mean] - df_features_g1[col_f2_mean]

        col_f1_max = f"{f1}_Flux_corrected_max"
        col_f2_max = f"{f2}_Flux_corrected_max"

        ratio_max = df_features_g1[col_f1_max] / (df_features_g1[col_f2_max] + 1e-6)
        diff_max = df_features_g1[col_f1_max] - df_features_g1[col_f2_max]

        temp_df = pd.DataFrame({
            f'{f1}_{f2}_mean_ratio': ratio_mean,
            f'{f1}_{f2}_mean_diff': diff_mean,
            f'{f1}_{f2}_max_ratio': ratio_max,
            f'{f1}_{f2}_max_diff': diff_max
        }, index=df_features_g1.index)

        new_features.append(temp_df)

    f_start, f_end = 'u', 'y'
    col_start = f"{f_start}_Flux_corrected_mean"
    col_end = f"{f_end}_Flux_corrected_mean"

    blue_red_ratio = df_features_g1[col_start] / (df_features_g1[col_end] + 1e-6)
    temp_extreme = pd.DataFrame({f'{f_start}_{f_end}_mean_ratio': blue_red_ratio}, index=df_features_g1.index)
    new_features.append(temp_extreme)

    df_colors = pd.concat(new_features, axis=1)
    df_colors = df_colors.replace([np.inf, -np.inf], 0).fillna(0)

    print(f"Hoàn thành Nhóm 3! Kích thước: {df_colors.shape}")
    return df_colors


BAZIN

In [ ]:
from scipy.optimize import least_squares
import warnings

# Tắt cảnh báo tính toán (overflow) để log đỡ rác
warnings.filterwarnings("ignore")

def bazin_func(params, t):
    """
    params: [A, B, t0, tau_rise, tau_fall]
    t: mảng thời gian
    """
    A, B, t0, tau_rise, tau_fall = params

    arg_fall = -(t - t0) / (tau_fall + 1e-5)
    arg_rise = -(t - t0) / (tau_rise + 1e-5)

    return A * (np.exp(arg_fall) / (1 + np.exp(arg_rise))) + B

In [ ]:
def residuals(params, t, flux, flux_err):
    """
    Hàm tính sai số để tối ưu hóa.
    Mục tiêu: Minimizing (Dữ liệu thực - Dữ liệu bazin) / Sai số đo
    """
    model = bazin_func(params, t)
    weights = 1.0 / (flux_err + 1e-5)
    return (flux - model) * weights

In [ ]:
def fit_bazin_one_series(time, flux, flux_err):
    """
    Hàm thực hiện fit cho 1 chuỗi thời gian (1 filter của 1 object)
    Trả về: [A, B, t0, tau_rise, tau_fall] và fit_error
    """

    # Chuyển về numpy array và sort theo thời gian
    idx = np.argsort(time)
    t = time[idx]
    f = flux[idx]
    e = flux_err[idx]

    # Chuẩn hóa thời gian để tính toán ổn định hơn (t bắt đầu từ 0)
    t_min = t.min()
    t_norm = t - t_min

    # Dự đoán tham số khởi tạo (Nếu đoán sai, least_squares sẽ không hội tụ)
    max_flux_idx = np.argmax(f)
    max_flux = f[max_flux_idx]

    guess_A = max_flux # Biên độ ~ Flux cao nhất
    guess_B = np.median(f[:3]) if len(f) > 3 else 0 # Nền ~ giá trị đoạn đầu
    guess_t0 = t_norm[max_flux_idx] # Đỉnh ~ thời điểm flux cao nhất
    guess_rise = 10.0 # Giả định ban đầu: tăng trong 10 giây
    guess_fall = 30.0 # Giả định band dầu: giảm trong 30 giây

    x0 = [guess_A, guess_B, guess_t0, guess_rise, guess_fall]

    # Giới hạn tham số
    # A > 0, Tau > 0. t0 nằm trong khoảng thời gian quan sát
    lower_bounds = [0, -np.inf, t_norm.min()-50, 0.1, 0.1]
    upper_bounds = [np.inf, np.inf, t_norm.max()+50, 200, 500]

    try:
        # Chạy tối ưu hóa
        res = least_squares(
            residuals,
            x0,
            args=(t_norm, f, e),
            bounds=(lower_bounds, upper_bounds),
            method='trf', # Trust Region Reflective
            loss='soft_l1' # Giảm ảnh hưởng của Outliers
        )

        final_params = res.x
        final_params[2] += t_min

        # Tính chi square (độ tốt của fit)
        chisq = np.sum(res.fun**2) / (len(f)-5)

        return list(final_params) + [chisq]
    except Exception:
        # Nếu lỗi, trả về NaN
        return [np.nan] * 6

In [ ]:
def generate_bazin_features(df_clean):
    print("Đang chạy Bazin Parametric Fitting....")

    features = []
    object_ids = df_clean['object_id'].unique()
    filters = ['u', 'g', 'r', 'i', 'z', 'y']

    # Lặp qua từng object. Để tối ưu, sau này có thể dùng jobLib Parallel
    cnt = 0
    total = len(object_ids)

    for obj_id in object_ids:
        df_obj = df_clean[df_clean['object_id'] == obj_id]

        obj_feats = {'object_id' : obj_id}

        for flt in filters:
            df_flt = df_obj[df_obj['Filter'] == flt]

            if len(df_flt) < 5:
                # Không đủ điểm để fit (cần tối thiểu 5 điểm cho 5 tham số)
                params = [np.nan] * 6
            else:
                params = fit_bazin_one_series(
                    df_flt['Time (MJD)'].values,
                    df_flt['Flux_corrected'].values,
                    df_flt['Flux_err_corrected'].values
                )
            
            # Lưu két quả: u_banzin_A, u_bazin_tau_rise, ...
            prefix = f"{flt}_bazin"
            obj_feats[f"{prefix}_A"] = params[0]
            obj_feats[f"{prefix}_B"] = params[1]
            obj_feats[f"{prefix}_t0"] = params[2]
            obj_feats[f"{prefix}_tau_rise"] = params[3]
            obj_feats[f"{prefix}_tau_fall"] = params[4]
            obj_feats[f"{prefix}_chisq"] = params[5]
        
        features.append(obj_feats)

        cnt += 1
        if cnt % 50 == 0:
            print(f"   -> Đã fit xong {cnt}/{total} vật thể.")
        
    df_bazin = pd.DataFrame(features)
    df_bazin.set_index('object_id', inplace=True)

    df_bazin = df_bazin.fillna(-1)

    print(f"Hoàn tất Bazin Fitting! Kích thước: {df_bazin.shape}")

    return df_bazin

# PROCESS TRAIN DATA

In [ ]:
from tqdm import tqdm

PROCESSED_DIR = "/MALLORN-Astronomical-Classification-Challenge/data/processed"
PROCESSED_TRAIN_FILE = "processed_train_full_lc.csv"

df_train_log = load_log(BASE_DIR, mode='train')
processed_chunks = []

print("--- Bắt đầu xử lý 20 splits---")

for i in range(1, 21):
    split_name = f"split_{i:02d}"
    print(f"Đang xử lý: {split_name} ({i}/20)...")

    df_clean = process_one_split(split_name, df_train_log, BASE_DIR, mode='train')
    if df_clean is None:
        continue

    df_g1 = generate_statistical_features(df_clean)
    df_g2 = generate_morphology_features(df_clean, df_g1)
    df_g3 = generate_color_features(df_g1)
    df_g4 = generate_bazin_features(df_clean)

    current_ids = df_g1.index
    df_train_log_tmp = df_train_log[df_train_log['object_id'].isin(current_ids)].copy()

    df_temp = pd.concat([df_g1, df_g2, df_g3], axis=1)
    df_temp = df_temp.join(df_g4, how='left')
    df_chunk_final = df_train_log_tmp.set_index('object_id').join(df_temp, how='inner')
    
    processed_chunks.append(df_chunk_final)

    print(f"   -> Xử lý xong {split_name}. Kích thước chunk: {df_chunk_final.shape}")

    # Dọn dẹp RAM
    del df_clean, df_g1, df_g2, df_g3, df_g4, df_chunk_final, df_train_log_tmp
    gc.collect()

print("Đang ghép nối tất cả các splits...")
if (len(processed_chunks) > 0):
    df_final_train_lc = pd.concat(processed_chunks, axis=0)
    print(f"Tổng kích thước: {df_final_train_lc.shape}")
    print(f"Tổng số TDE tìm thấy: {df_final_train_lc['target'].sum()}")

    # Lưu lại file trong processed
    save_path = os.path.join(PROCESSED_DIR, PROCESSED_TRAIN_FILE)
    df_final_train_lc.to_csv(save_path, index=True)
    print(f"Đã lưu file tại: {save_path}")


# PROCESS TEST DATA

In [ ]:
PROCESSED_TEST_FILE = "processed_test_full_lc.csv"

df_test_log = load_log(BASE_DIR, mode='test')
processed_chunks = []

print("--- Bắt đầu xử lý 20 splits---")

for i in range(1, 21):
    split_name = f"split_{i:02d}"
    print(f"Đang xử lý: {split_name} ({i}/20)...")

    df_clean = process_one_split(split_name, df_test_log, BASE_DIR, mode='test')
    if df_clean is None:
        continue

    df_g1 = generate_statistical_features(df_clean)
    df_g2 = generate_morphology_features(df_clean, df_g1)
    df_g3 = generate_color_features(df_g1)
    df_g4 = generate_bazin_features(df_clean)

    current_ids = df_g1.index
    df_test_log_tmp = df_test_log[df_test_log['object_id'].isin(current_ids)].copy()

    df_temp = pd.concat([df_g1, df_g2, df_g3], axis=1)
    df_temp = df_temp.join(df_g4, how='left')
    df_chunk_final = df_test_log_tmp.set_index('object_id').join(df_temp, how='inner')
    
    processed_chunks.append(df_chunk_final)

    print(f"   -> Xử lý xong {split_name}. Kích thước chunk: {df_chunk_final.shape}")

    # Dọn dẹp RAM
    del df_clean, df_g1, df_g2, df_g3, df_g4, df_chunk_final, df_test_log_tmp
    gc.collect()

print("Đang ghép nối tất cả các splits...")
if (len(processed_chunks) > 0):
    df_final_test_lc = pd.concat(processed_chunks, axis=0)

    # Lưu lại file trong processed
    save_path = os.path.join(PROCESSED_DIR, PROCESSED_TEST_FILE)
    df_final_test_lc.to_csv(save_path, index=True)
    print(f"Đã lưu file tại: {save_path}")


# FEATURE SELECTION

In [55]:
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from lightgbm import LGBMClassifier

class AdvancedFeatureSelector:
    def __init__(self, variance_thresh=0.0, correlation_thresh=0.98, max_features=140):
        self.var_thresh = variance_thresh
        self.corr_thresh = correlation_thresh
        self.max_features = max_features
        
        # Lưu danh sách các cột sẽ bỏ
        self.drop_cols_var = []
        self.drop_cols_corr = []
        self.drop_cols_model = []
        self.final_cols = []
        
    def fit(self, X, y):
        print("🛡️ [Smart Selector] Bắt đầu quy trình lọc features...")
        
        # --- BƯỚC 1: Variance Threshold ---
        # Tìm các cột có phương sai thấp (hằng số)
        selector_var = VarianceThreshold(threshold=self.var_thresh)
        selector_var.fit(X)
        # Lấy mask (True là giữ, False là bỏ)
        mask_var = selector_var.get_support()
        self.drop_cols_var = X.columns[~mask_var].tolist()
        
        X_v = X.loc[:, mask_var]
        print(f"   1. Variance: Loại {len(self.drop_cols_var)} cột hằng số. Còn lại: {X_v.shape[1]}")
        
        # --- BƯỚC 2: Correlation Filter ---
        # Tính ma trận tương quan
        print(f"   2. Correlation: Đang tính toán ma trận tương quan (> {self.corr_thresh})...")
        corr_matrix = X_v.corr().abs()
        
        # Chọn tam giác trên của ma trận
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        
        # Tìm các cột có correlation > threshold
        self.drop_cols_corr = [column for column in upper.columns if any(upper[column] > self.corr_thresh)]
        
        X_c = X_v.drop(columns=self.drop_cols_corr)
        print(f"      -> Loại bỏ {len(self.drop_cols_corr)} cột trùng lặp thông tin. Còn lại: {X_c.shape[1]}")
        
        # --- BƯỚC 3: Model-based Selection ---
        print(f"   3. Importance: Dùng LightGBM để chọn Top {self.max_features} features...")
        
        # Chỉ chạy bước này nếu số feature hiện tại > max_features
        if X_c.shape[1] > self.max_features:
            lgb = LGBMClassifier(n_estimators=200, learning_rate=0.05, random_state=42, n_jobs=-1, verbose=-1)
            lgb.fit(X_c, y)
            
            selector_model = SelectFromModel(lgb, max_features=self.max_features, prefit=True)
            mask_model = selector_model.get_support()
            
            self.final_cols = X_c.columns[mask_model].tolist()
            self.drop_cols_model = [c for c in X_c.columns if c not in self.final_cols]
            
            print(f"      -> Loại bỏ {len(self.drop_cols_model)} cột yếu. Giữ lại {len(self.final_cols)} features tinh túy.")
        else:
            self.final_cols = X_c.columns.tolist()
            print(f"      -> Số features hiện tại ({X_c.shape[1]}) <= max_features. Giữ nguyên.")
            
        return self

    def transform(self, X):
        # Chỉ việc lấy đúng danh sách cột cuối cùng
        return X[self.final_cols]

# MAIN

In [40]:
import numpy as np
import pandas as pd
import os
import re
import xgboost as xgb
import lightgbm as lgbm
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [ ]:
df_train = pd.read_csv("/MALLORN-Astronomical-Classification-Challenge/data/processed/processed_train_full_lc.csv")
df_test = pd.read_csv("/MALLORN-Astronomical-Classification-Challenge/data/processed/processed_test_full_lc.csv")

df_train = df_train.replace([np.inf, -np.inf], np.nan).fillna(-999)
df_test = df_test.replace([np.inf, -np.inf], np.nan).fillna(-999)

xgb_params = {
    'learning_rate': 0.051518799097782834,
    'max_depth': 10,
    'subsample': 0.9106527930168713,
    'colsample_bytree': 0.7028192063273316,
    'gamma': 0.6560553174292321,
    'reg_alpha': 8.328749198565307,
    'reg_lambda': 1.2457482664378638,
    'scale_pos_weight': 10.692975531517197,
    'n_estimators': 1500,
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'tree_method': 'hist',
    'random_state': 42,
    'n_jobs': -1,
    'early_stopping_rounds': 100
}

lgbm_params = {
    'learning_rate': 0.09910411868030085,
    'num_leaves': 51,
    'max_depth': 10,
    'subsample': 0.8473086396918044,
    'colsample_bytree': 0.6978315039075688,
    'reg_alpha': 7.089502551216929,
    'reg_lambda': 3.3773533645913636,
    'class_weight': 'balanced',
    'n_estimators': 1500,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

cat_params = {
    'learning_rate': 0.01844791176169218,
    'depth': 4,
    'l2_leaf_reg': 2.082185084558599,
    'random_strength': 5.209528399826417,
    'bagging_temperature': 0.5151147661811961,
    'scale_pos_weight': 7.369664685132736,
    'iterations': 1500,
    'loss_function': 'Logloss',
    'verbose': False,
    'random_seed': 42,
    'allow_writing_files': False,
    'early_stopping_rounds': 100
}

weakness_features = ['global_max', 'i_kurtosis', 'z_kurtosis', 
                     'y_Flux_corrected_min', 'r_Flux_corrected_mean', 
                     'z_y_mean_ratio', 'z_Flux_corrected_q75', 'y_bazin_A', 
                     'z_stetson_k', 'y_Flux_corrected_q75', 'g_Rise_slope', 
                     'y_Rise_slope', 'u_Rise_slope', 'i_Flux_corrected_mean', 
                     'i_Flux_corrected_std', 'g_Flux_corrected_count', 
                     'i_Flux_corrected_count', 'y_Rise_time', 
                     'g_Flux_corrected_mean', 'Z_err']

golden_features = [
    'Z', 'u_Flux_corrected_mean','g_Flux_corrected_std','r_Flux_corrected_max',
    'g_Flux_corrected_min','u_Flux_corrected_min','g_Flux_corrected_q05',
    'r_Flux_corrected_q25','u_Flux_corrected_q25','z_Flux_corrected_q25',
    'u_Flux_corrected_q75','z_Flux_corrected_q95','g_Flux_corrected_skew',
    'i_Flux_corrected_skew','r_Flux_corrected_skew','y_Flux_corrected_skew',
    'u_Flux_err_mean','i_Fall_time','r_Fall_time',
    'z_Fall_time','u_Fall_slope','z_Fall_slope',
    'u_percent_amplitude','r_percent_amplitude','z_percent_amplitude',
    'y_kurtosis','u_stetson_k','y_stetson_k','u_g_max_diff',
    'g_r_max_ratio','g_r_max_diff','r_i_mean_diff',
    'r_i_max_ratio','r_i_max_diff','i_z_max_diff',
    'z_y_mean_diff','u_bazin_B','u_bazin_tau_fall','u_bazin_chisq',
    'g_bazin_B','g_bazin_tau_fall','r_bazin_A','r_bazin_B','r_bazin_tau_rise',
    'r_bazin_tau_fall','r_bazin_chisq','i_bazin_tau_rise','i_bazin_tau_fall',
    'i_bazin_chisq','z_bazin_A','z_bazin_tau_rise','z_bazin_tau_fall',
    'z_bazin_chisq','y_bazin_chisq']

SEED = 42
SEEDS = [42, 2024, 123]
N_FOLDS = 5

In [56]:
drop_cols = ['object_id', 'SpecType', 'English Translation', 'split', 'target', 'prediction']
existing_drop = [c for c in drop_cols if c in df_train.columns]

X = df_train.drop(columns=existing_drop)
selector = AdvancedFeatureSelector(variance_thresh=0.0, correlation_thresh=0.98, max_features=120)
selector.fit(X, y)
X = selector.transform(X)
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

y = df_train['target']

train_cols = X.columns.tolist()
X_test = df_test[train_cols]

print(f" DỮ LIỆU ĐÃ SẴN SÀNG: X: {X.shape}, y:{y.sum()}/{len(y)}, X_test{X_test.shape}")

🛡️ [Smart Selector] Bắt đầu quy trình lọc features...
   1. Variance: Loại 1 cột hằng số. Còn lại: 176
   2. Correlation: Đang tính toán ma trận tương quan (> 0.98)...
      -> Loại bỏ 7 cột trùng lặp thông tin. Còn lại: 169
   3. Importance: Dùng LightGBM để chọn Top 120 features...
      -> Loại bỏ 115 cột yếu. Giữ lại 54 features tinh túy.
 DỮ LIỆU ĐÃ SẴN SÀNG: X: (3043, 54), y:148/3043, X_test(7135, 54)


In [ ]:
X_selected = X

golden_features = X_selected.columns.tolist() 
X_test_golden = df_test[golden_features]

# Biến tích lũy kết quả
final_test_pred_accumulated = np.zeros(len(X_test_golden))

print(f" BẮT ĐẦU HUẤN LUYỆN...")

for seed_idx, seed in enumerate(SEEDS):
    print(f"\n --- SEED {seed} ({seed_idx + 1}/{len(SEEDS)}) ---")
    
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=seed)
    
    # Reset biến OOF cho seed này
    oof_xgb = np.zeros(len(X_selected))
    oof_lgbm = np.zeros(len(X_selected))
    oof_cat = np.zeros(len(X_selected))
    
    seed_test_xgb = np.zeros(len(X_test_golden))
    seed_test_lgbm = np.zeros(len(X_test_golden))
    seed_test_cat = np.zeros(len(X_test_golden))
    
    # Update random_state
    xgb_params['random_state'] = seed
    lgbm_params['random_state'] = seed
    cat_params['random_seed'] = seed

    # Training Loop
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_selected, y)):
        X_tr, X_val = X_selected.iloc[train_idx], X_selected.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # 1. XGBoost
        model_xgb = xgb.XGBClassifier(**xgb_params)
        model_xgb.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        oof_xgb[val_idx] = model_xgb.predict_proba(X_val)[:, 1]
        seed_test_xgb += model_xgb.predict_proba(X_test_golden)[:, 1] / N_FOLDS

        # 2. LightGBM
        model_lgbm = lgbm.LGBMClassifier(**lgbm_params)
        model_lgbm.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], callbacks=[lgbm.early_stopping(100, verbose=False)])
        oof_lgbm[val_idx] = model_lgbm.predict_proba(X_val)[:, 1]
        seed_test_lgbm += model_lgbm.predict_proba(X_test_golden)[:, 1] / N_FOLDS

        # 3. CatBoost
        train_pool = Pool(X_tr, y_tr)
        val_pool = Pool(X_val, y_val)
        model_cat = CatBoostClassifier(**cat_params)
        model_cat.fit(train_pool, eval_set=val_pool, verbose=False)
        oof_cat[val_idx] = model_cat.predict_proba(X_val)[:, 1]
        seed_test_cat += model_cat.predict_proba(X_test_golden)[:, 1] / N_FOLDS
        
        print(f"   Fold {fold+1} Done.")

    # Tìm trọng số tối ưu cho Seed này
    def get_best_f1(y_true, y_prob):
        best_s, best_t = 0, 0
        for th in np.arange(0.2, 0.8, 0.01):
            s = f1_score(y_true, (y_prob >= th).astype(int))
            if s > best_s: best_s, best_t = s, th
        return best_s, best_t

    # Grid search nhỏ cho trọng số
    weights_to_try = [
        (1, 3, 1), # Trọng số chiến thắng của bạn vừa rồi
        (1, 1, 1), (2, 2, 1), (1, 2, 1), (1, 1, 2), (2, 1, 2)
    ]
    
    best_seed_f1 = 0
    best_w = (1, 1, 1)
    
    for w1, w2, w3 in weights_to_try:
        blend = (w1*oof_xgb + w2*oof_lgbm + w3*oof_cat) / (w1+w2+w3)
        s, _ = get_best_f1(y, blend)
        if s > best_seed_f1:
            best_seed_f1 = s
            best_w = (w1, w2, w3)
            
    print(f"    Best Seed {seed}: F1={best_seed_f1:.4f} | Weights={best_w}")
    
    # Dự báo Test của Seed này
    w1, w2, w3 = best_w
    seed_final_pred = (w1*seed_test_xgb + w2*seed_test_lgbm + w3*seed_test_cat) / (w1+w2+w3)
    
    final_test_pred_accumulated += seed_final_pred

# --- KẾT QUẢ CUỐI CÙNG ---
final_avg_prob = final_test_pred_accumulated / len(SEEDS)

FINAL_TH = 0.50

submission = pd.DataFrame({
    'object_id': df_test['object_id'],
    'prediction': (final_avg_prob > FINAL_TH).astype(int)
})

filename = "submission_golden_multiseed.csv"
submission.to_csv(filename, index=False)
print(f"🎉 ĐÃ TẠO FILE: {filename}")

🚀 BẮT ĐẦU CHIẾN DỊCH: GOLDEN FEATURES x MULTI-SEED...

🌱 --- SEED 42 (1/3) ---
   Fold 1 Done.
   Fold 2 Done.
   Fold 3 Done.
   Fold 4 Done.
   Fold 5 Done.
   🏆 Best Seed 42: F1=0.6026 | Weights=(1, 3, 1)

🌱 --- SEED 2024 (2/3) ---
   Fold 1 Done.
   Fold 2 Done.
   Fold 3 Done.
   Fold 4 Done.
   Fold 5 Done.
   🏆 Best Seed 2024: F1=0.5737 | Weights=(2, 1, 2)

🌱 --- SEED 123 (3/3) ---
   Fold 1 Done.
   Fold 2 Done.
   Fold 3 Done.
   Fold 4 Done.
   Fold 5 Done.
   🏆 Best Seed 123: F1=0.6207 | Weights=(1, 1, 2)
🎉 ĐÃ TẠO FILE: submission_golden_multiseed.csv
👉 File này chứa sức mạnh của 54 Features tinh túy x 3 Seeds x 3 Models!


In [61]:
print(f"BẮT ĐẦU QUÁ TRÌNH HUẤN LUYỆN....")

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

oof_xgb = np.zeros(len(X))
oof_lgbm = np.zeros(len(X))
oof_cat = np.zeros(len(X))

test_xgb = np.zeros(len(X_test))
test_lgbm = np.zeros(len(X_test))
test_cat = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"   -> Đang chạy Fold {fold+1}/{N_FOLDS}...")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model_xgb = xgb.XGBClassifier(**xgb_params)
    model_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    oof_xgb[val_idx] = model_xgb.predict_proba(X_val)[:, 1]
    test_xgb += model_xgb.predict_proba(X_test)[:, 1] / N_FOLDS

    model_lgbm = lgbm.LGBMClassifier(**lgbm_params)
    model_lgbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgbm.early_stopping(100, verbose=False)])
    oof_lgbm[val_idx] = model_lgbm.predict_proba(X_val)[:, 1]
    test_lgbm += model_lgbm.predict_proba(X_test)[:, 1] / N_FOLDS

    train_pool = Pool(X_train, y_train)
    val_pool = Pool(X_val, y_val)
    model_cat = CatBoostClassifier(**cat_params)
    model_cat.fit(train_pool, eval_set=val_pool, verbose=False)
    oof_cat[val_idx] = model_cat.predict_proba(val_pool)[:, 1]
    test_cat += model_cat.predict_proba(X_test)[:, 1] / N_FOLDS
    
def get_best_f1(y_true, y_prob):
    best_s, best_t = 0, 0
    for th in np.arange(0.1, 0.9, 0.01):
        s = f1_score(y_true, (y_prob >= th).astype(int))
        if s > best_s: best_s, best_t = s, th
    return best_s, best_t

print("Kết quả huấn luyện:")
f1_xgb, th_xgb = get_best_f1(y, oof_xgb)
print(f"  - XGBoost   : F1 = {f1_xgb:.4f} (Thresh={th_xgb:.2f})")
f1_lgbm, th_lgbm = get_best_f1(y, oof_lgbm)
print(f"  - LightGBM  : F1 = {f1_lgbm:.4f} (Thresh={th_lgbm:.2f})")
f1_cat, th_cat = get_best_f1(y, oof_cat)
print(f"  - CatBoost  : F1 = {f1_cat:.4f} (Thresh={th_cat:.2f})")   

BẮT ĐẦU QUÁ TRÌNH HUẤN LUYỆN....
   -> Đang chạy Fold 1/5...
   -> Đang chạy Fold 2/5...
   -> Đang chạy Fold 3/5...
   -> Đang chạy Fold 4/5...
   -> Đang chạy Fold 5/5...
Kết quả huấn luyện:
  - XGBoost   : F1 = 0.5871 (Thresh=0.39)
  - LightGBM  : F1 = 0.5839 (Thresh=0.58)
  - CatBoost  : F1 = 0.6012 (Thresh=0.45)


In [58]:
best_f1 = 0
best_w = (1, 1, 1)
best_th = 0.5

weights_to_try = [
    (1, 1, 1), (2, 1, 1), (1, 2, 1), (1, 1, 2),
    (2, 2, 1), (2, 1, 2), (1, 2, 2),
    (3, 1, 1), (1, 3, 1), (1, 1, 3)
]

for w1, w2, w3 in weights_to_try:
    blend = (w1*oof_xgb + w2*oof_lgbm + w3*oof_cat) / (w1+w2+w3)
    s, th = get_best_f1(y, blend)
    if s > best_f1:
        best_f1 = s
        best_w = (w1, w2, w3)
        best_th = th

print(f"KẾT QUẢ ENSEMBLE:")
print(f"  - Trọng số: XGB={best_w[0]}, LGBM={best_w[1]}, CAT={best_w[2]}")
print(f"  - F1-Score: {best_f1:.4f}")
print(f"  - Threshold: {best_th:.2f}")    

KẾT QUẢ ENSEMBLE:
  - Trọng số: XGB=1, LGBM=3, CAT=1
  - F1-Score: 0.6026
  - Threshold: 0.52


In [59]:
w1, w2, w3 = best_w
total_w = w1 + w2 + w3
y_pred_blend = (w1 * test_xgb + w2 * test_lgbm + w3 * test_cat) / total_w
submission = pd.DataFrame({
    'object_id': df_test['object_id'],
    'prediction': (y_pred_blend > best_th).astype(int)
})

sub_name = "submission.csv"
submission.to_csv(sub_name, index=False)
print(f"\n HOÀN TẤT! Đã tạo file {sub_name}.")


 HOÀN TẤT! Đã tạo file submission.csv.
